In [1]:
import json
import pandas as pd
import uuid
import pprint as pp
pprint = pp.PrettyPrinter(indent=4).pprint
import requests
import pysondb
import xml.etree.ElementTree as ET

In [62]:
url = "http://localhost:3000/api/project/list_papers"
headers = {
    'Content-Type': 'application/json'
}
data = {
    # 'type': "saved", # or toread
    'type': "toread", # or toread
    'project_id': 148702791823144807,
}


response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.text)

# add_project("robotaics")

[
  {
    "abstract": "With quantum computing technologies nearing the era of commercialization and\nquantum supremacy, machine learning (ML) appears as one of the promising\n\"killer\" applications. Despite significant effort, there has been a disconnect\nbetween most quantum ML proposals, the needs of ML practitioners, and the\ncapabilities of near-term quantum devices to demonstrate quantum enhancement in\nthe near future. In this contribution to the focus collection on \"What would\nyou do with 1000 qubits?\", we provide concrete examples of intractable ML tasks\nthat could be enhanced with near-term devices. We argue that to reach this\ntarget, the focus should be on areas where ML researchers are struggling, such\nas generative models in unsupervised and semi-supervised learning, instead of\nthe popular and more tractable supervised learning techniques. We also\nhighlight the case of classical datasets with potential quantum-like\nstatistical correlations where quantum models cou

In [23]:




def get_semantic_scholar_info(arxiv_id):
    base_url = "https://api.semanticscholar.org/v1/paper/arXiv:"
    
    # Fetch data from Semantic Scholar
    response = requests.get(base_url + arxiv_id)
    data = response.json()

    # Return citation count and impact score
    # Note: Semantic Scholar doesn't have a generic "impact score", but citations can be a metric of impact.
    return len(data.get("citations", []))



def get_paper_base_data(paper_id):
    base_url = "http://export.arxiv.org/api/query?"
    query = "id_list=" + paper_id
    
    # Send a GET request to the arXiv API
    response = requests.get(base_url + query)
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()
    
    # Parse the XML response
    root = ET.fromstring(response.text)
    
    # Extract information from XML
    ns = {'default': 'http://www.w3.org/2005/Atom'}
    entry = root.find('default:entry', ns)
    
    title = entry.find('default:title', ns).text.strip()
    authors = [author.find('default:name', ns).text for author in entry.findall('default:author', ns)]
    abstract = entry.find('default:summary', ns).text.strip()
    year = entry.find('default:published', ns).text.split('-')[0]
    
    # Return the extracted information in the desired format
    return {
        "paper_id": paper_id,
        "title": title,
        "authors": authors,
        "abstract": abstract,
        "year": year,
        "citations": get_semantic_scholar_info(paper_id)
    }

# Test
paper_id = "1710.03599"
# paper_id = "2101.12345"
pprint(get_paper_base_data(paper_id))


{   'abstract': 'Quantum computing allows for the potential of significant '
                'advancements in\n'
                'both the speed and the capacity of widely used machine '
                'learning techniques.\n'
                'Here we employ quantum algorithms for the Hopfield network, '
                'which can be used\n'
                'for pattern recognition, reconstruction, and optimization as '
                'a realization of a\n'
                'content-addressable memory system. We show that an '
                'exponentially large network\n'
                'can be stored in a polynomial number of quantum bits by '
                'encoding the network\n'
                'into the amplitudes of quantum states. By introducing a '
                'classical technique for\n'
                'operating the Hopfield network, we can leverage quantum '
                'algorithms to obtain a\n'
                'quantum computational complexity that is logarit

In [18]:
paper_in_project = {
    "paper_id": "1710.03599",
    "project_id": "1234567890",
    "rating": 1,
    "annotations": {},
    "engagement": {
        "view_count": 0,
        "view_duration": 0,
    },
    "saved": True,
    "read": False
}

In [2]:
projects = pysondb.db.getDb("data/projects")
projPapers = pysondb.db.getDb("data/project_papers")


In [4]:
def add_project(name):
    project = {
        "name": name,
        "papers": [],
        "imgUrl": "/AI.jpeg"
    }
    return projects.add(project) # returns id

In [5]:
add_project("Computational Neuroscience")

utf-8


175233565509294414

In [20]:
def add_paper_to_project(project_id, paper_id): # paper_id is arxiv id
    project_paper = {
        "project_id": project_id,
        "paper_id": paper_id,
        "rating": 1,
        "annotations": {},
        "engagement": {
            "click_count": 0,
            "view_duration": 0,
        },
        "saved": False,
        "read": False
    }
    id = projPapers.add(project_paper)

    old_papers = projects.getById(project_id)["papers"]
    old_papers.append(id)

    projects.updateById(project_id, {"papers": old_papers})

In [21]:
def update_engagement(metric, value, project_id, paper_id):
    db_paper_id = projPapers.getByQuery({"paper_id": paper_id, "project_id": project_id})[0]["id"]
    
    if metric == "click":
        upd_engagement = projPapers.getById(db_paper_id)["engagement"]
        upd_engagement["click_count"] += 1
        projPapers.updateById(db_paper_id, {"engagement": upd_engagement})
    elif metric == "view":
        upd_engagement = projPapers.getById(db_paper_id)["engagement"]
        upd_engagement["view_duration"] += value
        projPapers.updateById(db_paper_id, {"engagement": upd_engagement})
    elif metric == "read":
        projPapers.updateById(db_paper_id, {"read": value})
    elif metric == "save":
        projPapers.updateById(db_paper_id, {"saved": value})


In [63]:
papers = pysondb.db.getDb("data/papers")
projects = pysondb.db.getDb("data/projects")
projPapers = pysondb.db.getDb("data/project_papers")

papers.deleteAll()
projects.deleteAll()
projPapers.deleteAll()

# testing: this is intital data


# add quantum
project = {
    "name": "Quantum Physics",
    "papers": [],
    "imgUrl": "/QuantumPhysics.jpeg"
}
qn_id = projects.add(project)


paper_ids = ['1709.02779',
 '1611.09347',
 '1611.09347v2',
 '1708.09757',
 '1709.02779',
 '1707.08561v1',
 '1708.09757v1',
 '2208.08068',
 '1710.03599v1',
 '1908.04480v2']

# proj_id = add_project("Sample Project")

for paper_id in paper_ids:
    papers.add(get_paper_base_data(paper_id))
    add_paper_to_project(qn_id, paper_id)

    

update_engagement("click", 1, qn_id, paper_id)
update_engagement("view", 15, qn_id, paper_id)
update_engagement("view", 15, qn_id, paper_id)
update_engagement("read", True, qn_id, paper_id)
update_engagement("save", True, qn_id, paper_id)
update_engagement("save", True, qn_id, paper_ids[0])
update_engagement("save", True, qn_id, paper_ids[2])
update_engagement("save", True, qn_id, paper_ids[3])
update_engagement("save", True, qn_id, paper_ids[4])
update_engagement("read", True, qn_id, paper_ids[4])
update_engagement("read", True, qn_id, paper_ids[2])


utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


In [32]:
projPapers.getAll()

148702791823144807
9007199254740991

[{'project_id': 148702791823144807,
  'paper_id': '1709.02779',
  'rating': 1,
  'annotations': {},
  'engagement': {'click_count': 0, 'view_duration': 0},
  'saved': True,
  'read': True,
  'id': 178494889177972764},
 {'project_id': 148702791823144807,
  'paper_id': '1611.09347',
  'rating': 1,
  'annotations': {},
  'engagement': {'click_count': 0, 'view_duration': 0},
  'saved': False,
  'read': False,
  'id': 239451734285630063},
 {'project_id': 148702791823144807,
  'paper_id': '1611.09347v2',
  'rating': 1,
  'annotations': {},
  'engagement': {'click_count': 0, 'view_duration': 0},
  'saved': True,
  'read': True,
  'id': 754822796031261480},
 {'project_id': 148702791823144807,
  'paper_id': '1708.09757',
  'rating': 1,
  'annotations': {},
  'engagement': {'click_count': 0, 'view_duration': 0},
  'saved': True,
  'read': False,
  'id': 207245952752736323},
 {'project_id': 148702791823144807,
  'paper_id': '1709.02779',
  'rating': 1,
  'annotations': {},
  'engagement': {'click

In [53]:
request = {
    # 'type': "saved", # or toread
    'type': "toread", # or toread
    'project_id': 148702791823144807,
}

def list_papers():
    proj_papers = projPapers.getByQuery({"project_id": request["project_id"]})

    if request["type"] == "saved":
        proj_papers = list(filter(lambda x: x["saved"] == True & x["read"] == True, proj_papers))
    elif request["type"] == "toread":
        new = []
        for paper in proj_papers:
            if paper["saved"] == True and paper["read"] == False:
                new.append(paper)
        proj_papers = new


    # reformat and get paper data
    output = []

    for paper in proj_papers:
        paper_data = papers.getByQuery({"paper_id": paper["paper_id"]})[0]
        
        del paper_data["paper_id"]
        del paper_data["citations"]
        paper_data["journal"] = "bullshit"
        paper_data["authors"] = ', '.join(paper_data["authors"])

        print(paper['read'], paper['saved'])

        output.append(paper_data)



    return output

list_papers()

False True


[{'title': 'Opportunities and challenges for quantum-assisted machine learning in\n  near-term quantum computers',
  'authors': 'Alejandro Perdomo-Ortiz, Marcello Benedetti, John Realpe-Gómez, Rupak Biswas',
  'abstract': 'With quantum computing technologies nearing the era of commercialization and\nquantum supremacy, machine learning (ML) appears as one of the promising\n"killer" applications. Despite significant effort, there has been a disconnect\nbetween most quantum ML proposals, the needs of ML practitioners, and the\ncapabilities of near-term quantum devices to demonstrate quantum enhancement in\nthe near future. In this contribution to the focus collection on "What would\nyou do with 1000 qubits?", we provide concrete examples of intractable ML tasks\nthat could be enhanced with near-term devices. We argue that to reach this\ntarget, the focus should be on areas where ML researchers are struggling, such\nas generative models in unsupervised and semi-supervised learning, instead

In [64]:
from metaphor_python import Metaphor
METAPHOR_API_KEY = "2c14596c-15cd-42e6-b969-d4717a6f60ee"
metaphor = Metaphor(api_key=METAPHOR_API_KEY)

In [78]:
def extract_arxiv_id(url):
    end = url.split('/')[-1].split('.')
    try:
        return end[0] + '.' + end[1]
    except:
        return end[0]

In [73]:
# today = datetime.date.today()
# one_year_ago = today - datetime.timedelta(days=365)
user_query = "quantum computing" 

# Metaphor API call
response = metaphor.search(user_query, num_results=10, 
                        include_domains=["arxiv.org"],
                        # start_crawl_date=str(one_year_ago), 
                        use_autoprompt=True,)

In [82]:
ids = [extract_arxiv_id(r.url) for r in response.results]

for id in ids:
    if papers.getByQuery({"paper_id": id}) == []:
        papers.add(get_paper_base_data(id))
    add_paper_to_project(qn_id, id) # todo change to project id

utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


In [85]:
ids

['1312.4455',
 '1608.00263v2',
 '1801.00862v3',
 '2209.06841',
 '1801.00862',
 '1206.0785v1',
 '1312.4455v1',
 '1501.00011v1',
 '1709.06678',
 '1710.05867v2']

In [89]:
 # reformat and get paper data
output = []

for paper in ids:
    paper_data = papers.getByQuery({"paper_id": paper})[0]
    
    del paper_data["paper_id"]
    del paper_data["citations"]
    paper_data["journal"] = "Unknown"
    paper_data["authors"] = ', '.join(paper_data["authors"])

    # print(paper['read'], paper['saved'])

    output.append(paper_data)

output

[{'title': 'The universe as quantum computer',
  'authors': 'Seth Lloyd',
  'abstract': 'This article reviews the history of digital computation, and investigates\njust how far the concept of computation can be taken. In particular, I address\nthe question of whether the universe itself is in fact a giant computer, and if\nso, just what kind of computer it is. I will show that the universe can be\nregarded as a giant quantum computer. The quantum computational model of the\nuniverse explains a variety of observed phenomena not encompassed by the\nordinary laws of physics. In particular, the model shows that the the quantum\ncomputational universe automatically gives rise to a mix of randomness and\norder, and to both simple and complex systems.',
  'year': '2013',
  'id': 205079175082933215,
  'journal': 'Unknown'},
 {'title': 'Characterizing Quantum Supremacy in Near-Term Devices',
  'authors': 'Sergio Boixo, Sergei V. Isakov, Vadim N. Smelyanskiy, Ryan Babbush, Nan Ding, Zhang Jiang,

In [97]:
url = "http://localhost:3000/api/search"
args = {
    "query": "quantum physics",
    "project_id": "754468843766675212",
}


response = requests.get(url, params=args)

print(response.text)

# add_project("robotaics")

[
  {
    "abstract": "The properties which give quantum mechanics its unique character - unitarity,\ncomplementarity, non-commutativity, uncertainty, nonlocality - derive from the\nalgebraic structure of Hermitian operators acting on the wavefunction in\ncomplex Hilbert space. Because of this, the wavefunction cannot be shown to\ndescribe an ensemble of deterministic states where uncertainty simply reflects\na lack of knowledge about which ensemble member describes reality. This has led\nto endless debates about the ontology of quantum mechanics.\n  Here we derive these same quantum properties from number theoretic attributes\nof trigonometric functions applied to an explicitly ensemble-based\nrepresentation of discretised complex Hilbert states. To avoid fine-tuning, the\nmetric on state space must be $p$-adic rather than Euclidean where $1/p$\ndetermines the fineness of the discretisation. This hints at both the existence\nof an underpinning fractal state-space geometry onto which s

In [9]:
import random

In [10]:
random.uniform(0, 1)

0.7797139512452512

In [11]:
entries = projPapers.getAll()

for entry in entries:
    projPapers.updateById(entry["id"], {"rating": random.uniform(0, 1)})

AttributeError: 'list' object has no attribute 'updateById'